In [1]:
# TODO: Добавить NER. Иначе сейчас лемматизатор из Гай Ричи делаей гай ричать.
# TODO: Использовать TF-IDF, чтобы удалить слабочастотные слова

# Library import

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [24]:
from pathlib import Path
import re
from typing import List, Union

import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
import plotly
import plotly.express as px
import pymorphy2
import pyLDAvis.gensim_models

[nltk_data] Downloading package punkt to /home/semenyutaav-
[nltk_data]     ml/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/semenyutaav-
[nltk_data]     ml/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.9/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [4]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
# import pyLDAvis.gensim_models.prepare
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.9/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/usr/local/lib/python3.9/dist-packages/matplotlib/__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/usr/local/lib/python3.9/dist-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/usr/local/lib/python3.9/dist-packages/matplotlib/__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/usr/local/lib/python3.9/dist-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packa

In [5]:
import collections

In [6]:
LANG = 'russian'
PUNCTUATION_PATTERN = r'[`“”Δδ•©.,;–\-+—»«><%:?!*=()\'\]\[\\/]+'

In [7]:
# tokenized_sent = ['—', '–', '»', '.', 'lol', '»lol', '.lol', '\'', '\'Павел\'']
# for token_idx, token in enumerate(tokenized_sent):
#     punc_patterns = re.findall(PUNCTUATION_PATTERN, token.strip().lower())
#     print(punc_patterns)

# # remove_punct(tokenized_sent)
# # punc_patterns

# Text loading functions

In [8]:
PATHS = [
    Path('../data/kinopoisk/...A-zori-zdes\'-tihie/0.txt'),
    Path('../data/kinopoisk/1+1/0.txt'),
    Path('../data/kinopoisk/12-razgnevannyh-muzhchin/0.txt'),
    Path('../data/kinopoisk/Bojtsovskij-klub/0.txt'),
    Path('../data/kinopoisk/Bol\'shoj-kush/0.txt'),
    Path('../data/kinopoisk/Brat/0.txt'),
    Path('../data/kinopoisk/Dostuchat\'sja-do-nebes/0.txt'),
    Path('../data/kinopoisk/Dusha/0.txt'),
    Path('../data/kinopoisk/Dzhentl\'meny/0.txt'),
    Path('../data/kinopoisk/Dzhentl\'meny-udachi/0.txt'),
    
    Path('../data/politics/rbc/Bajden-sdal-otritsatel\'nyj-test-na-COVID-19-posle-zarazhenija-Psaki.txt'),
    Path('../data/politics/rbc/Bajden-zajavil-Zelenskomu-o-zhelanii-posetit\'-Ukrainu.txt'),
    Path('../data/politics/rbc/Baku-vyzval-vremennogo-poverennogo-Rossii-iz-za-slov-Zhirinovskogo.txt'),
    Path('../data/politics/rbc/Belyj-dom-ispravil-dolju-evakuirovannyh-iz-Afganistana-v-rechi-Bajdena.txt'),
    Path('../data/politics/rbc/Belyj-dom-nazval-uslovija-dlja-vstuplenija-Ukrainy-v-NATO.txt'),
    Path('../data/politics/rbc/Blinken-i-novaja-glava-MID-Germanii-Berbok-obsudili-«rossijskuju-agressiju».txt'),
    Path('../data/politics/rbc/Blinken-prigrozil-Rossii-sanktsijami,-kotoryh-ne-primenjali-ranee.txt'),
    Path('../data/politics/rbc/Bloomberg-soobschil-o-nezhelanii-chlena-NATO-voevat\'-s-Rossiej-za-Ukrainu.txt'),
    Path('../data/politics/rbc/Bloomberg-soobschil-o-skorom-odobrenii-novyh-sanktsij-ES-protiv-Belorussii.txt'),
    Path('../data/politics/rbc/Britanija-prigrozila-Iranu-posledstvijami-za-napadenie-na-tanker.txt'),
    
    Path('../data/psychohournal/"Prognosticheskie"-sny-v-kontekste-obrazno-simvolicheskogo-soznanija.txt'),
    Path('../data/psychohournal/«Bojus\'-obraschat\'sja-k-psihologu».-Kak-nadumannye-strahi-lishajut-nas-professional\'noj-pomoschi.txt'),
    Path('../data/psychohournal/«Unizhennost\'-i-oskorblennost\'»-kak-cherta-lichnosti:-fenomenologicheskij-analiz-pozitsii-zhertvy.txt'),
    Path('../data/psychohournal/5-sposobov-razvit\'-svoju-unikal\'nost\'.txt'),
    Path('../data/psychohournal/6-priznakov-zdorovyh-otnoshenij.txt'),
    Path('../data/psychohournal/6-sovetskih-zagadok-na-logiku-i-vnimatel\'nost\'-v-kartinkah.txt'),
    Path('../data/psychohournal/7-navykov-psihicheski-ustojchivyh-ljudej.txt'),
    Path('../data/psychohournal/Anna-Bezljudnaja-psiholog:-snjatie-porchi-metodami-gipnoterapii.txt'),
    Path('../data/psychohournal/Art-terapija-vo-vremja-samoizoljatsii---podderzhka-i-istselenie.txt'),
    Path('../data/psychohournal/Biblioterapija-dissotsiativnyh-sostojanij.txt'),
    
    Path('../data/receipts/povarenok/"L\'ezhskij"-salat-so-struchkovoj-fasol\'ju.txt'),
    Path('../data/receipts/povarenok/"Oliv\'e"-s-mussom-iz-avokado.txt'),
    Path('../data/receipts/povarenok/Baklazhan-farshirovannyj-tvorozhnym-syrom.txt'),
    Path('../data/receipts/povarenok/Baklazhan,-zapravlennyj-limonno-medovoj-karamel\'ju.txt'),
    Path('../data/receipts/povarenok/Baklazhany,-zapechennye-v-orehovoj-panirovke.txt'),
    Path('../data/receipts/povarenok/Fruktovyj-salat-s-jogurtom-i-pechen\'em.txt'),
    Path('../data/receipts/povarenok/Belaja-ryba-v-hrustjaschej-orehovoj-panirovke.txt'),
    Path('../data/receipts/povarenok/Blinchiki-s-povidlom-iz-chernosliva.txt'),
    Path('../data/receipts/povarenok/Bulochki-kak-puh.txt'),
    Path('../data/receipts/povarenok/Desert-"-Novogodnjaja-sladost\'".txt'),
         
    Path('../data/russiadiscovery/-Provereno-na-sebe:-Severnyj-Kavkaz.txt'),
    Path('../data/russiadiscovery/Provereno-na-sebe:-Adygeja.txt'),
    Path('../data/russiadiscovery/Provereno-na-sebe:-Antarktida.txt'),
    Path('../data/russiadiscovery/Provereno-na-sebe:-bajkal\'skij-led.txt'),
    Path('../data/russiadiscovery/Provereno-na-sebe:-Chechnja-i-Dagestan.txt'),
    Path('../data/russiadiscovery/Provereno-na-sebe:-Chuhlomskie-terema.txt'),
    Path('../data/russiadiscovery/Provereno-na-sebe:-Dombaj.txt'),
    Path('../data/russiadiscovery/Provereno-na-sebe:-ekspeditsija-na-Jamal.txt'),
    Path('../data/russiadiscovery/Provereno-na-sebe:-Gornyj-Altaj.txt'),
    Path('../data/russiadiscovery/Provereno-na-sebe:-Juzhnyj-Ural.txt'),
    
]

In [9]:
def load_txt(path: Path) -> str:
    """Loads text from txt file.
    
    Args:
        path: A path to a txt file.
    
    Return:
        A txt in a string.
    """
    text = ''
    with open(path, 'r', encoding='utf-8') as fin:
        text = ''.join(fin)
    return text

def load_texts(paths: List[Path]) -> List[str]:
    """Loads a sample of texts in a list.
    
    Args:
        paths: A list of paths to files to be loaded.
    
    Return:
        A list with loaded texts.
    """
    texts = []
    for path in paths:
        text = load_txt(path)
        texts.append(text)
    return texts

def load_texts_by_dir(path: str) -> List[str]:
    """
    """
    paths = sorted(Path(path).glob('**/*.txt'))
    texts = load_texts(paths)
    return texts, paths

# Text preprocessing functions

In [10]:
def tokenize_texts(text_list: List[str]) -> List[List[str]]:
    """Tokenizes a list of texts.
    
    Args:
        text_matrix: A matrix with words of texts.
    
    Return:
        A matrix with tokenized texts.
    """
    tokenized_matrix = []
    
    for text in text_list:
        tokenized_matrix.append(word_tokenize(text, language=LANG))
        
    return tokenized_matrix

In [11]:
word_tokenize('Я ненавижу, лол, \'Павел\' дворфа.', language=LANG)

['Я', 'ненавижу', ',', 'лол', ',', "'Павел", "'", 'дворфа', '.']

In [12]:
def lowercase_matrix(text_matrix: List[List[str]]) -> List[List[str]]:
    """Lower case of matrix words.
    
    Args:
        text_matrix: A matrix with words of texts.
    
    Return:
        A matrix with words in a lowercase.
    """
    lowercased_matrix = []
    
    for text in text_matrix:
        text_list = []
        for token in text:
            text_list.append(token.lower())
            
        lowercased_matrix.append(text_list)
        
    return lowercased_matrix

In [13]:
def remove_stopwords(tokenized_matrix: List[List[str]]) -> List[List[str]]:
    """Removes stop words from a matrix.
    
    Args:
        tokenized_matrix: A matrix with words of texts.
    
    Return:
        A matrix without stop words.
    """
    stop_words = stopwords.words(LANG)
    
    indices_to_remove_dict = {}
    for sent_idx, tokenized_sent in enumerate(tokenized_matrix):
        for token_idx, token in enumerate(tokenized_sent):
            if token in stop_words:
                if sent_idx not in indices_to_remove_dict:
                    indices_to_remove_dict[sent_idx] = []
                indices_to_remove_dict[sent_idx].append(token)

    for sent_idx in indices_to_remove_dict:
        for token in indices_to_remove_dict[sent_idx]:
            del tokenized_matrix[sent_idx][tokenized_matrix[sent_idx].index(token)]
    return tokenized_matrix

In [14]:
def remove_punct(tokenized_matrix: List[List[str]]) -> List[List[str]]:
    """Removes punctuation words and digits and 1-symbol-words from a matrix.
    
    Args:
        tokenized_matrix: A matrix with words of texts.
    
    Return:
        A matrix without punctuation words.
    """
    indices_to_remove_dict = {}
    for sent_idx, tokenized_sent in enumerate(tokenized_matrix):
        for token_idx, token in enumerate(tokenized_sent):
            punc_patterns = re.findall(PUNCTUATION_PATTERN, token.strip().lower())
            if (len(punc_patterns) > 0 and punc_patterns[0] == token.strip().lower()) or \
                (token.isdigit()) or \
                    (len(token) == 1):
                if sent_idx not in indices_to_remove_dict:
                    indices_to_remove_dict[sent_idx] = []
                indices_to_remove_dict[sent_idx].append(token)

    for sent_idx in indices_to_remove_dict:
        for token in indices_to_remove_dict[sent_idx]:
            del tokenized_matrix[sent_idx][tokenized_matrix[sent_idx].index(token)]
    return tokenized_matrix

In [15]:
def remove_word_edge_punct(token: str) -> str:
    """"""
    init_token = token
    while True:
        punc_patterns = re.findall(PUNCTUATION_PATTERN, token[0])
        if len(punc_patterns) > 0:
            token = token[1:]
        else:
            break
            
    while True:
        punc_patterns = re.findall(PUNCTUATION_PATTERN, token[-1])
        if len(punc_patterns) > 0:
            token = token[:-1]
        else:
            break
            
    return token
            

def remove_words_edge_punct(text_matrix: List[List[str]]) -> List[List[str]]:
    """Lower case of matrix words.
    
    Args:
        text_matrix: A matrix with words of texts.
    
    Return:
        A matrix with words in a lowercase.
    """
    lowercased_matrix = []
    
    for text in text_matrix:
        text_list = []
        for token in text:
            text_list.append(remove_word_edge_punct(token))
            
        lowercased_matrix.append(text_list)
        
    return lowercased_matrix

In [16]:
#r'[`.,;–\-—»«%:?!*=()\']+'

In [17]:
# morph = pymorphy2.MorphAnalyzer()
# morph.parse('Ричи')[0].normal_form

In [18]:
def lemmatize_matrix(tokenized_matrix: List[List[str]]) -> List[List[str]]:
    """Lemmatizes matrix texts via pymorphy2.
    
    Args:
        tokenized_matrix: A matrix with words of texts.
    
    Return:
        A matrix with lemmatized words.
    """
    morph = pymorphy2.MorphAnalyzer()

    lemmatized_matrix = []
    for sent_idx, tokenized_sent in enumerate(tokenized_matrix):
        lemmatized_matrix.append([])
        for token_idx, token in enumerate(tokenized_sent):
            lemmatized_matrix[sent_idx].append(morph.parse(token)[0].normal_form)

    return lemmatized_matrix

In [19]:
def build_vocabulary(tokenized_texts, max_size=1000000, max_doc_freq=1, min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    doc_n = 0
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    word_count = {word: cnt for word, cnt in word_counts.items() if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    return word_count

In [20]:
def filter_by_freq(tokenized_texts, max_size=1000000, max_doc_freq=1, min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    doc_n = 0
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    word_count = word_counts
    
    indices_to_remove_dict = {}
    for sent_idx, tokenized_sent in enumerate(tokenized_texts):
        for token_idx, token in enumerate(tokenized_sent):
            if word_count[token] < min_count or word_count[token] / doc_n > max_doc_freq:
                if sent_idx not in indices_to_remove_dict:
                    indices_to_remove_dict[sent_idx] = []
                indices_to_remove_dict[sent_idx].append(token)

    for sent_idx in indices_to_remove_dict:
        for token in indices_to_remove_dict[sent_idx]:
            del tokenized_texts[sent_idx][tokenized_texts[sent_idx].index(token)]
            
    return tokenized_texts

# Text processing via fuctions above

In [1]:
from pathlib import Path
import sys

ROOT_DIR = Path('../').resolve()
if ROOT_DIR not in sys.path:
    sys.path.append(str(ROOT_DIR))

from components.downloader.downloader_lib import download_packages
from components.loader.loader_lib import load_texts_by_dir
from components.tokenizer.tokenizer_lib import tokenize_texts
from components.filter.filter_lib import lowercase_matrix

In [8]:
download_packages()

[nltk_data] Downloading package punkt to /home/semenyutaav-
[nltk_data]     ml/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/semenyutaav-
[nltk_data]     ml/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
texts, paths = load_texts_by_dir('../data')
print(len(texts))
texts[0]

8898


'Еще одно крутое место в Бангкоке, которое официально закрыто, но в этом-то и вся прелесть (можно найти способ и проникнуть).Когда-то здесь был торговый центр New World Mall. Когда он стал заброшкой, внутри начала скапливаться вода, на которой отлично увеличивали свою популяцию комары. Это сильно не понравилось местным жителям и они решили запустить в воду рыб. Сейчас это просто amazing огромный аквариум, в котором плавают какие-то крутые и экзотические виды, среди них есть огромные и не очень рыбины разных цветов. Мне их очень жаль на самом деле.Сейчас торговый центр закрыт, официально пройти в него нельзя. Но если вы сильно захотите, то я же не смогу вас остановить. Правда рыб там больше нет.Чтобы было еще более загадочно, я не покажу вам фотографию здания снаружи. Оно находится в Бангкоке, недалеко от Коасан роуд, в крайне туристическом месте. Вот координаты на карте Google: 13.762172, 100.498286 И кто ищет — тот найдет.by Ольга Салий. Копирование материала запрещено.Другие интересн

In [20]:
!cat ../components/tokenizer/tokenizer_lib.py

from typing import List

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

from components.common.constants import LANG


def tokenize_texts(text_list: List[str]) -> List[List[str]]:
    """Tokenizes a list of texts.

    Args:
        text_matrix: A matrix with words of texts.

    Return:
        A matrix with tokenized texts.
    """
    tokenized_matrix = []

    for text in text_list:
        tokenized_matrix.append(word_tokenize(text, language=LANG))

    return tokenized_matrix


In [21]:
# texts = load_texts(PATHS)
# print(len(texts))
# texts[0]

In [25]:
tokenized_matrix = tokenize_texts(texts)
tokenized_matrix[0]

['Еще',
 'одно',
 'крутое',
 'место',
 'в',
 'Бангкоке',
 ',',
 'которое',
 'официально',
 'закрыто',
 ',',
 'но',
 'в',
 'этом-то',
 'и',
 'вся',
 'прелесть',
 '(',
 'можно',
 'найти',
 'способ',
 'и',
 'проникнуть',
 ')',
 '.Когда-то',
 'здесь',
 'был',
 'торговый',
 'центр',
 'New',
 'World',
 'Mall',
 '.',
 'Когда',
 'он',
 'стал',
 'заброшкой',
 ',',
 'внутри',
 'начала',
 'скапливаться',
 'вода',
 ',',
 'на',
 'которой',
 'отлично',
 'увеличивали',
 'свою',
 'популяцию',
 'комары',
 '.',
 'Это',
 'сильно',
 'не',
 'понравилось',
 'местным',
 'жителям',
 'и',
 'они',
 'решили',
 'запустить',
 'в',
 'воду',
 'рыб',
 '.',
 'Сейчас',
 'это',
 'просто',
 'amazing',
 'огромный',
 'аквариум',
 ',',
 'в',
 'котором',
 'плавают',
 'какие-то',
 'крутые',
 'и',
 'экзотические',
 'виды',
 ',',
 'среди',
 'них',
 'есть',
 'огромные',
 'и',
 'не',
 'очень',
 'рыбины',
 'разных',
 'цветов',
 '.',
 'Мне',
 'их',
 'очень',
 'жаль',
 'на',
 'самом',
 'деле.Сейчас',
 'торговый',
 'центр',
 'закрыт'

In [29]:
tokenized_matrix = lowercase_matrix(tokenized_matrix)
tokenized_matrix[0]

['еще',
 'одно',
 'крутое',
 'место',
 'в',
 'бангкоке',
 ',',
 'которое',
 'официально',
 'закрыто',
 ',',
 'но',
 'в',
 'этом-то',
 'и',
 'вся',
 'прелесть',
 '(',
 'можно',
 'найти',
 'способ',
 'и',
 'проникнуть',
 ')',
 '.когда-то',
 'здесь',
 'был',
 'торговый',
 'центр',
 'new',
 'world',
 'mall',
 '.',
 'когда',
 'он',
 'стал',
 'заброшкой',
 ',',
 'внутри',
 'начала',
 'скапливаться',
 'вода',
 ',',
 'на',
 'которой',
 'отлично',
 'увеличивали',
 'свою',
 'популяцию',
 'комары',
 '.',
 'это',
 'сильно',
 'не',
 'понравилось',
 'местным',
 'жителям',
 'и',
 'они',
 'решили',
 'запустить',
 'в',
 'воду',
 'рыб',
 '.',
 'сейчас',
 'это',
 'просто',
 'amazing',
 'огромный',
 'аквариум',
 ',',
 'в',
 'котором',
 'плавают',
 'какие-то',
 'крутые',
 'и',
 'экзотические',
 'виды',
 ',',
 'среди',
 'них',
 'есть',
 'огромные',
 'и',
 'не',
 'очень',
 'рыбины',
 'разных',
 'цветов',
 '.',
 'мне',
 'их',
 'очень',
 'жаль',
 'на',
 'самом',
 'деле.сейчас',
 'торговый',
 'центр',
 'закрыт'

In [25]:
tokenized_matrix = remove_stopwords(tokenized_matrix)
tokenized_matrix[0]

['одно',
 'крутое',
 'место',
 'бангкоке',
 ',',
 'которое',
 'официально',
 'закрыто',
 ',',
 'этом-то',
 'вся',
 'прелесть',
 '(',
 'найти',
 'способ',
 'проникнуть',
 ')',
 '.когда-то',
 'торговый',
 'центр',
 'new',
 'world',
 'mall',
 '.',
 'стал',
 'заброшкой',
 ',',
 'внутри',
 'начала',
 'скапливаться',
 'вода',
 ',',
 'которой',
 'отлично',
 'увеличивали',
 'популяцию',
 'комары',
 '.',
 'это',
 'сильно',
 'понравилось',
 'местным',
 'жителям',
 'решили',
 'запустить',
 'воду',
 'рыб',
 '.',
 'это',
 'просто',
 'amazing',
 'огромный',
 'аквариум',
 ',',
 'котором',
 'плавают',
 'какие-то',
 'крутые',
 'экзотические',
 'виды',
 ',',
 'среди',
 'огромные',
 'очень',
 'рыбины',
 'разных',
 'цветов',
 '.',
 'очень',
 'жаль',
 'самом',
 'деле.сейчас',
 'торговый',
 'центр',
 'закрыт',
 ',',
 'официально',
 'пройти',
 '.',
 'сильно',
 'захотите',
 ',',
 'смогу',
 'остановить',
 '.',
 'правда',
 'рыб',
 'нет.чтобы',
 'загадочно',
 ',',
 'покажу',
 'фотографию',
 'здания',
 'снаружи',

In [26]:
tokenized_matrix = remove_punct(tokenized_matrix)
tokenized_matrix = remove_words_edge_punct(tokenized_matrix)
tokenized_matrix[0]

['одно',
 'крутое',
 'место',
 'бангкоке',
 'которое',
 'официально',
 'закрыто',
 'этом-то',
 'вся',
 'прелесть',
 'найти',
 'способ',
 'проникнуть',
 'когда-то',
 'торговый',
 'центр',
 'new',
 'world',
 'mall',
 'стал',
 'заброшкой',
 'внутри',
 'начала',
 'скапливаться',
 'вода',
 'которой',
 'отлично',
 'увеличивали',
 'популяцию',
 'комары',
 'это',
 'сильно',
 'понравилось',
 'местным',
 'жителям',
 'решили',
 'запустить',
 'воду',
 'рыб',
 'это',
 'просто',
 'amazing',
 'огромный',
 'аквариум',
 'котором',
 'плавают',
 'какие-то',
 'крутые',
 'экзотические',
 'виды',
 'среди',
 'огромные',
 'очень',
 'рыбины',
 'разных',
 'цветов',
 'очень',
 'жаль',
 'самом',
 'деле.сейчас',
 'торговый',
 'центр',
 'закрыт',
 'официально',
 'пройти',
 'сильно',
 'захотите',
 'смогу',
 'остановить',
 'правда',
 'рыб',
 'нет.чтобы',
 'загадочно',
 'покажу',
 'фотографию',
 'здания',
 'снаружи',
 'оно',
 'находится',
 'бангкоке',
 'недалеко',
 'коасан',
 'роуд',
 'крайне',
 'туристическом',
 'мес

In [27]:
lemmatized_matrix = lemmatize_matrix(tokenized_matrix)
lemmatized_matrix[0]

/usr/local/lib/python3.9/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.9/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.9/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.9/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

['один',
 'крутой',
 'место',
 'бангкок',
 'который',
 'официально',
 'закрытый',
 'это-то',
 'весь',
 'прелесть',
 'найти',
 'способ',
 'проникнуть',
 'когда-то',
 'торговый',
 'центр',
 'new',
 'world',
 'mall',
 'стать',
 'заброшка',
 'внутри',
 'начало',
 'скапливаться',
 'вода',
 'который',
 'отлично',
 'увеличивать',
 'популяция',
 'комар',
 'это',
 'сильно',
 'понравиться',
 'местный',
 'житель',
 'решить',
 'запустить',
 'вода',
 'рыба',
 'это',
 'просто',
 'amazing',
 'огромный',
 'аквариум',
 'который',
 'плавать',
 'какой-то',
 'крутой',
 'экзотический',
 'вид',
 'среди',
 'огромный',
 'очень',
 'рыбина',
 'разный',
 'цвет',
 'очень',
 'жаль',
 'сам',
 'деле.сейчас',
 'торговый',
 'центр',
 'закрытый',
 'официально',
 'пройти',
 'сильно',
 'захотеть',
 'смочь',
 'остановить',
 'правда',
 'рыба',
 'нет.чтоба',
 'загадочно',
 'показать',
 'фотография',
 'здание',
 'снаружи',
 'оно',
 'находиться',
 'бангкок',
 'недалеко',
 'коасан',
 'роуд',
 'крайне',
 'туристический',
 'мест

In [28]:
filtered_matrix = filter_by_freq(lemmatized_matrix, max_doc_freq=0.43, min_count=5)
filtered_matrix[0]

['один',
 'крутой',
 'место',
 'бангкок',
 'который',
 'официально',
 'закрытый',
 'это-то',
 'весь',
 'прелесть',
 'найти',
 'способ',
 'проникнуть',
 'когда-то',
 'торговый',
 'центр',
 'new',
 'стать',
 'заброшка',
 'внутри',
 'начало',
 'вода',
 'который',
 'отлично',
 'увеличивать',
 'популяция',
 'комар',
 'это',
 'сильно',
 'понравиться',
 'местный',
 'житель',
 'решить',
 'запустить',
 'вода',
 'рыба',
 'это',
 'просто',
 'огромный',
 'аквариум',
 'который',
 'плавать',
 'какой-то',
 'крутой',
 'экзотический',
 'вид',
 'среди',
 'огромный',
 'очень',
 'разный',
 'цвет',
 'очень',
 'жаль',
 'сам',
 'торговый',
 'центр',
 'закрытый',
 'официально',
 'пройти',
 'сильно',
 'захотеть',
 'смочь',
 'остановить',
 'правда',
 'рыба',
 'показать',
 'фотография',
 'здание',
 'снаружи',
 'оно',
 'находиться',
 'бангкок',
 'недалеко',
 'крайне',
 'туристический',
 'место',
 'координата',
 'карта',
 'google',
 'искать',
 'ольга',
 'салий',
 'копирование',
 'материал',
 'интересный',
 'пост',

In [29]:
dict1 = build_vocabulary(lemmatized_matrix, max_doc_freq=1, min_count=0) # max_doc_freq=0.43
sorted_tuples = sorted(dict1.items(), key=lambda item: item[1])
# print(sorted_tuples)  # [(1, 1), (3, 4), (2, 9)]
sorted_dict = {k: v for k, v in sorted_tuples}
print(len(sorted_dict))
# print(sorted_dict['ещё'])

11118


In [30]:
sorted_tuples = sorted(dict1.items(), key=lambda item: item[1], reverse=True)
# print(sorted_tuples)  # [(1, 1), (3, 4), (2, 9)]
sorted_dict = {k: v for k, v in sorted_tuples}
# print(len(sorted_dict))
# print(sorted_dict['ещё'])

In [31]:
sorted_dict

{'весь': 3822,
 'самый': 3810,
 'просто': 3696,
 'каждый': 3600,
 'стать': 3498,
 'первый': 3492,
 'новый': 3348,
 'хороший': 3288,
 'это': 3249,
 'который': 3246,
 'сделать': 3186,
 'жизнь': 3126,
 'хотеть': 3102,
 'также': 3090,
 'дать': 2978,
 'место': 2922,
 'делать': 2886,
 'работа': 2856,
 'говорить': 2832,
 'дело': 2796,
 'должный': 2796,
 'фильм': 2784,
 'нужно': 2772,
 'наш': 2736,
 'свой': 2673,
 'день': 2670,
 'сказать': 2646,
 'знать': 2628,
 'смотреть': 2622,
 'иметь': 2616,
 'часть': 2598,
 'несколько': 2580,
 'друг': 2556,
 'мир': 2556,
 'история': 2538,
 'любой': 2502,
 'именно': 2496,
 'сторона': 2442,
 'человек': 2388,
 'момент': 2376,
 'ваш': 2364,
 'поэтому': 2328,
 'например': 2322,
 'посмотреть': 2310,
 'случай': 2304,
 'время': 2295,
 'стоить': 2280,
 'вид': 2274,
 'главный': 2268,
 'казаться': 2244,
 'слово': 2238,
 'больший': 2208,
 'найти': 2202,
 'работать': 2202,
 'некоторый': 2196,
 'мочь': 2193,
 'вопрос': 2178,
 'образ': 2166,
 'получить': 2166,
 'видеть'

# LDA

In [32]:
# Create Dictionary 
id2word = corpora.Dictionary(filtered_matrix)  
# Create Corpus 
# texts = lemmatized_matrix  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in filtered_matrix]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 7), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 3), (22, 2), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 3), (33, 1), (34, 3), (35, 1), (36, 2), (37, 1), (38, 3), (39, 2), (40, 1), (41, 2), (42, 6), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 2), (58, 2), (59, 1), (60, 3), (61, 1), (62, 2), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 3), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 4), (87, 2), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 5), (97, 2), (98, 1), (99, 1), (100, 1), (101, 2), (102, 2), (103, 1), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1)

In [33]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('15-20', 1),
  ('5,00', 1),
  ('email', 2),
  ('google', 1),
  ('mail.ru', 1),
  ('new', 1),
  ('автор', 2),
  ('адрес', 1),
  ('аквариум', 1),
  ('бангкок', 7),
  ('бат', 1),
  ('бетонный', 1),
  ('бкк', 1),
  ('благодарный', 1),
  ('близкий', 1),
  ('блог', 1),
  ('блогер', 1),
  ('больший', 2),
  ('бомж', 1),
  ('буква', 1),
  ('быстро', 1),
  ('быть', 3),
  ('ваш', 2),
  ('вердикт', 1),
  ('версия', 1),
  ('весь', 2),
  ('взгляд', 1),
  ('вид', 1),
  ('видеть', 1),
  ('видимо', 2),
  ('видно', 1),
  ('внимательный', 1),
  ('внутри', 3),
  ('внутрь', 1),
  ('вода', 3),
  ('возможно', 1),
  ('вообще', 2),
  ('впечатляющий', 1),
  ('вроде', 3),
  ('всё', 2),
  ('всё-таки', 1),
  ('второй', 2),
  ('вход', 6),
  ('вызывать', 1),
  ('выяснить', 1),
  ('гид', 1),
  ('главное', 2),
  ('главный', 1),
  ('глаз', 1),
  ('глубокий', 1),
  ('гораздо', 1),
  ('гореть', 1),
  ('горячий', 1),
  ('гость', 1),
  ('давно', 1),
  ('далёкий', 1),
  ('два', 1),
  ('дверь', 2),
  ('действительно', 2),

In [34]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

#  Compute Perplexity

print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
print('Initializing Coherence Mode')
coherence_model_lda = CoherenceModel(model=lda_model, texts=filtered_matrix, dictionary=id2word, corpus=corpus, coherence='u_mass')
print('Getting coherence score')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Perplexity:  -10.358855154360388
Initializing Coherence Mode
Getting coherence score

Coherence Score:  -1.7795853899803273


In [35]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.047*"добавить" + 0.030*"минута" + 0.026*"чеснок" + 0.023*"соль" + '
  '0.023*"лук" + 0.022*"нарезать" + 0.021*"продукт" + 0.020*"масло" + '
  '0.018*"салат" + 0.017*"огонь"'),
 (1,
  '0.016*"путешествие" + 0.015*"место" + 0.012*"наш" + 0.012*"маршрут" + '
  '0.011*"поездка" + 0.011*"поезд" + 0.010*"самый" + 0.010*"самолёт" + '
  '0.009*"день" + 0.008*"дорога"'),
 (2,
  '0.030*"масло" + 0.026*"вода" + 0.024*"использовать" + 0.022*"сковорода" + '
  '0.017*"форма" + 0.013*"поверхность" + 0.012*"пока" + 0.011*"жидкость" + '
  '0.011*"добавлять" + 0.010*"должный"'),
 (3,
  '0.050*"ваш" + 0.014*"например" + 0.012*"хороший" + 0.012*"должный" + '
  '0.010*"сделать" + 0.010*"найти" + 0.010*"место" + 0.010*"также" + '
  '0.009*"помочь" + 0.009*"каждый"'),
 (4,
  '0.115*"нажать" + 0.056*"удалить" + 0.049*"сша" + 0.044*"раздел" + '
  '0.034*"опция" + 0.030*"пакет" + 0.030*"меню" + 0.024*"битый" + '
  '0.020*"поместить" + 0.019*"потребоваться"'),
 (5,
  '0.019*"говорить" + 0.016*"сказать"

In [36]:
# len(filtered_matrix[:876]+filtered_matrix[1400:])

In [37]:
# filtered_matrix[960]

In [38]:
#  Compute Perplexity
time_start = time.time()
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
print('Initializing Coherence Mode')
coherence_model_lda = CoherenceModel(
    model=lda_model, texts=filtered_matrix, 
    dictionary=id2word, corpus=corpus, coherence='u_mass')
print('Getting coherence score')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print(time.time() - time_start)

NameError: name 'time' is not defined

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

In [ ]:
def save_tokenized_texts(tokenized_texts: List[List[str]], paths: List[Union[str, Path]]):
    """"""
    assert len(tokenized_texts) == len(paths)
    zipped = zip(tokenized_texts, paths)
    for text, path in zipped:
        save_path = str(path).replace('data/', 'data/processed/')
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(save_path, "w") as file:
            file.write(' '.join(text))

In [ ]:
save_tokenized_texts(filtered_matrix, paths)

In [ ]:
assert len(filtered_matrix) == len(paths)

In [ ]:
save_tokenized_texts(filtered_matrix, paths)